# Introduction

NexDatawork is an AI data agent for data engineering and analytics without writing code.

## Prerequisites

 - Node.js
 - Supabase
 - OpenAI


 Before starting your work install all the required tools:

In [ ]:
!pip install langchain_openai
!pip install langchain
!pip install langchain_experimental

Import all the required libraries

In [ ]:
import os
import pandas as pd
from IPython.display import Markdown, HTML, display
from langchain_openai import AzureChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import gradio as gr
import pandas as pd
import io
import contextlib
from sqlalchemy import (Engine,create_engine,MetaData,Table,Column,String,Integer,Float,insert,inspect,text)
from huggingface_hub import login

To access AzureOpenAI models you'll need to create an Azure account, create a deployment of an Azure OpenAI model, get the name and endpoint for your deployment, get an Azure OpenAI API key and install the langchain-openai integration package.

Replace the placeholders with the actual values.

In [ ]:
os.environ["AZURE_OPENAI_ENDPOINT"] = "INSERT THE OPENAI ENDPOINT"
os.environ["AZURE_OPENAI_API_KEY"] = "INSERT YOUR OPENAI API KEY

To set up the Azure OpenAI model choose the name for ```AZURE_DEPLOYMENT_NAME``` and insert ```AZURE_API_VERSION``` (the latest supported version can be found here: https://learn.microsoft.com/en-us/azure/ai-services/openai/reference).

In [ ]:
# Load your Azure environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT_NAME = "gpt-4.1"  # 👈 Change if needed
AZURE_API_VERSION = "2025-01-01-preview"  # 👈 Use your correct version

# Define Azure LLM with streaming enabled
model = AzureChatOpenAI(
    openai_api_version=AZURE_API_VERSION,
    azure_deployment=AZURE_DEPLOYMENT_NAME,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


Here you can create a prompt for your model.

In [ ]:
# Prompt
CSV_PROMPT_PREFIX = """
Set pandas to show all columns.
Get the column names and infer data types.
Then attempt to answer the question using multiple methods.
Please provide only the Python code required to perform the action, and nothing else.
"""

CSV_PROMPT_SUFFIX = """
- Try at least 2 different methods of calculation or filtering.
- Reflect: Do they give the same result?
- After performing all necessary actions and analysis with the dataframe, return the answer in clean **Markdown**, include summary table if needed.
- Include **Execution Recommendation** and **Web Insight** in the final Markdown.
- Always conclude the final Markdown with:

### Final Answer

Your conclusion here.

---

### Explanation

Mention specific columns you used.
Please provide only the Python code required to perform the action, and nothing else until the final Markdown output.
"""

SQL_PROMPT = """
Write code in SQL for extracting data based on the user's question
"""

The following block is responsible for the logic of the agent and the output that it produces.

In [ ]:
# Replace this with your actual LLM setup
# Example:
# from langchain_openai import AzureChatOpenAI
# model = AzureChatOpenAI(...)

def sql_code(files, question):
    try:
        dfs = [pd.read_csv(f.name) for f in files]
    except Exception as e:
        return f"❌ Could not read CSVs: {e}", ""
    try:
        agent = create_pandas_dataframe_agent(
        llm=model,
        df=df,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        allow_dangerous_code=True,
        handle_parsing_errors=True,  # 👈 this is the fix
      )




# --- Agent Logic ---
def ask_agent(files, question):
    try:
        dfs = [pd.read_csv(f.name) for f in files]
        df = pd.concat(dfs, ignore_index=True)
    except Exception as e:
        return f"❌ Could not read CSVs: {e}", ""

    try:
        agent = create_pandas_dataframe_agent(
        llm=model,
        df=df,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        allow_dangerous_code=True,
        handle_parsing_errors=True,  # 👈 this is the fix
    )


        full_prompt = CSV_PROMPT_PREFIX + question + CSV_PROMPT_SUFFIX

        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            result = agent.invoke(full_prompt)
        trace = buffer.getvalue()
        output = result["output"]


        return output, trace

    except Exception as e:
        return f"❌ Agent error: {e}", ""


The block below deals with creating SQL code.

In [ ]:
os.environ["HUGGINGFACE_TOKEN"] = "ENTER YOUR HUGGING FACE TOKEN"
login(token = "ENTER YOUR ACCESS TOKEN")

In [ ]:
#function create_db receives a dictionary with table names as a key and tables as values
def create_db(files: dict) -> Engine:
    engine = create_engine("sqlite:///:memory:")

    with engine.begin() as connection:
      for name,table in zip(files.keys(),files.values()):
        table.to_sql(name,connection,if_exists="replace",index=False)

    inspection = inspect(engine)
    table_names = inspection.get_table_names()
    return engine


#Function that returns the query
@tool
def sql_engine(query: str, engine: Engine) -> str:
    """Allows you to perform SQL queries on the tables available in the database engine.

    First, examine the names of tables in the database in the engine.

    After understanding the table structure, choose the most appropriate table and create a correct SQL query for it.

    If there is no appropriate answer for the question based on the available tables, state that there might be a mistake in the request or the data.

    Returns a string representation of the result.

    Args:
        query: The query to perform. This should be correct SQL.

        engine: Engine that provides connection to the database

    """
    output = ""
    try:
        with engine.connect() as con:
            rows = con.execute(text(query))
            result_rows = rows.fetchall()
            column_names = rows.keys()
            output += ", ".join(column_names) + "\n"
            for row in result_rows:
                output += ", ".join(map(str, row)) + "\n"
    except Exception as e:
        output = f"Error executing query: {e}"
    return query

The next section creates the GUI and starts the programme.

In [ ]:
# --- Gradio UI ---
with gr.Blocks(
    css="""
    body, .gradio-container {
        background: #ffffff !important;
        color: #1f2937 !important;
        font-family: 'Segoe UI', sans-serif;
    }

    #title {
        color: #1f2937 !important;
        font-size: 2rem;
        font-weight: 600;
        text-align: center;
        padding-top: 20px;
        padding-bottom: 10px;
    }

    .gr-box, .gr-input, .gr-output, .gr-markdown, .gr-textbox, .gr-file, textarea, input {
        background: rgba(0, 0, 0, 0.04) !important;
        border: 1px solid rgba(0, 0, 0, 0.1);
        border-radius: 12px !important;
        color: #1f2937 !important;
    }

    textarea::placeholder, input::placeholder {
        color: rgba(31, 41, 55, 0.6) !important;
    }

    button {
        background: rgba(0, 0, 0, 0.07) !important;
        color: #1f2937 !important;
        border: 1px solid rgba(0, 0, 0, 0.15) !important;
        border-radius: 8px !important;
    }

    button:hover {
        background: rgba(0, 0, 0, 0.15) !important;
    }
    """
) as demo:

    gr.Markdown("<h2 id='title'>📊 NexDatawork Data Agent</h2>")

    with gr.Column():
        result_display = gr.Markdown(label="📌 Report Output (Markdown)")
        trace_display = gr.Textbox(label="🛠️ Data Agent Reasoning - Your Explainable Agent", lines=20)

    with gr.Row(equal_height=True):
        file_input = gr.File(label="📁 Upload CSV(s)", file_types=[".csv"], file_count="multiple")
        with gr.Column():
            question_input = gr.Textbox(
        label="💬 Ask Your Data",
        placeholder="e.g., What is the trend for revenue over time?",
        lines=2
    )
            sql_code = gr.Textbox(label="SQL code",placeholder='SQL code will appear here',lines=2)


    ask_button = gr.Button("💡 Analyze")

    ask_button.click(
        fn=ask_agent,
        inputs=[file_input, question_input],
        outputs=[result_display, trace_display]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1b7614b12579466c16.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
